<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Ejemplo de API con BERT

## 1 - Instalar dependencias (ya sea en el colab o en su PC/servidor)

In [ ]:
!pip install transformers --quiet

In [ ]:
# Descargar los pesos entrenados de BERT desde un gogle drive (es la forma más rápida)
# NOTA: No hay garantía de que estos links perduren, en caso de que no estén
# disponibles, se pueden obtener del entrenamiento de BERT de la clase anterior
!curl -L -o 'bert_weights.h5' 'https://drive.google.com/u/0/uc?id=1ILoVmLK3IFMOZiWEkqvqSmnHF7a--3h2&export=download&confirm=t'

In [ ]:
%%writefile app.py
# -------------------------------------------------------------------
# TODO ESTO PODRÍA ESTAR EN OTRO ARCHIVO
# ------------------------------------------------------------------
import tensorflow as tf
import numpy as np

from transformers import file_utils
from transformers import BertTokenizer, TFBertModel, BertConfig
print("Donde se almacena el cache?:", file_utils.default_cache_path)


class MyBertModel():
    def __init__(self, model_weights_path):
        self.max_length = 140
        self.class_names = ['negative', 'neutral', 'positive']
        self.model_weights_path = model_weights_path
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.build_model()

    def build_model(self):
        output_shape = len(self.class_names)       

        bert_model = TFBertModel.from_pretrained("bert-base-uncased")

        input_ids = tf.keras.layers.Input(shape=(self.max_length,), dtype=tf.int32, name='input_ids')
        attention_mask = tf.keras.layers.Input((self.max_length,), dtype=tf.int32, name='attention_mask')

        # Get the pooled_output (embedding que representa toda la entrada)
        output = bert_model([input_ids, attention_mask])[1] 

        # We can also add dropout as regularization technique:
        output = tf.keras.layers.Dropout(rate=0.2)(output)

        # Se puede agregar más capas Densas en el medio si se desea

        # Provide number of classes to the final layer:
        output = tf.keras.layers.Dense(output_shape, activation='softmax')(output)

        # Final model:
        self.model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
        self.model.load_weights(self.model_weights_path)


    def predict(self, input_text):

        tf_batch = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_length, # truncates if len(s) > max_length
            return_token_type_ids=False,
            return_attention_mask=True,
            padding="max_length", # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True,
            return_tensors='tf'
        )

        X_ensayo = [tf_batch['input_ids'],  
                    tf_batch['attention_mask']]

        y_prob_ensayo = self.model.predict(X_ensayo)
        y_prob = np.argmax(y_prob_ensayo, axis=1)
        class_predicted = self.class_names[int(y_prob)]
        print("Input:", input_text)
        print("Clasificacion:", class_predicted)
        return class_predicted

# -------------------------------------------------------------------
# CREAR NUESTRA API con Flask
# ------------------------------------------------------------------
import traceback
from flask import Flask, jsonify
# Crear el server Flask
app = Flask(__name__)

# Crear el modelo que utilizaremos en la API
modelo = MyBertModel('bert_weights.h5')

@app.route("/")
def index():
    msg = '''Para poder solicitar una prediccion debe acceder al endpoint:
            /predict/<input_text>
            Debe reemplazar <input_text> en el explorador por el texto
            que desea ingresar al modelo
            '''
    return msg

@app.route("/predict/<input_text>")
def predict(input_text):
    # Siempre es recomendable colocar nuestro
    # código entre try except para que el servidor
    # no se caiga si llega a fallar algo
    try:
        return modelo.predict(input_text)
    except:
        # En caso de falla, retornar el mensaje de error
        return jsonify({'trace': traceback.format_exc()})


if __name__ == '__main__':
    # Lanzar server
    app.run(host="127.0.0.1", port=5000, debug=True)

In [ ]:
# Exponer el puerto 5000 de colab al exterior
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
# Lanzar la aplicacion
!python app.py